In [1]:
import numpy as np
import pandas as pd
import re
from os.path import join

from tools import *

In [2]:
## rename EXTRACT_FEATURES to DIMENSION_REDUCTION
read_prefix = "/data/MIMIC3/"
write_prefix = "/data/liu/LDA"

In [1]:
# test re replace remove string remain number
import re
# l=["diag_0","diag_1"]
# [re.sub("[^0-9]", "",s) for s in l]

['0', '1']

## Read Diagnoses and Prescriptions Data

In [3]:
diag_df = read_data(join(read_prefix,"DIAGNOSES_ICD"))
diag_df.head()

,ROW_ID,SUBJECT_ID,HADM_ID,SEQ_NUM,ICD9_CODE
0,1297,109,172335,1.0,40301
1,1298,109,172335,2.0,486
2,1299,109,172335,3.0,58281
3,1300,109,172335,4.0,5855
4,1301,109,172335,5.0,4254


In [4]:
pres_df = read_data(join(read_prefix,"PRESCRIPTIONS"),dtype={'NDC':str})
pres_df.head()

/home/liu/anaconda3/envs/py36/lib/python3.6/site-packages/IPython/core/interactiveshell.py:3214: DtypeWarning: Columns (11) have mixed types. Specify dtype option on import or set low_memory=False.
  if (yield from self.run_code(code, result)):


,ROW_ID,SUBJECT_ID,HADM_ID,ICUSTAY_ID,STARTDATE,ENDDATE,DRUG_TYPE,DRUG,DRUG_NAME_POE,DRUG_NAME_GENERIC,FORMULARY_DRUG_CD,GSN,NDC,PROD_STRENGTH,DOSE_VAL_RX,DOSE_UNIT_RX,FORM_VAL_DISP,FORM_UNIT_DISP,ROUTE
0,2214776,6,107064,NaN,2175-06-11 00:00:00,2175-06-12 00:00:00,MAIN,Tacrolimus,Tacrolimus,Tacrolimus,TACR1,021796,00469061711,1mg Capsule,2,mg,2,CAP,PO
1,2214775,6,107064,NaN,2175-06-11 00:00:00,2175-06-12 00:00:00,MAIN,Warfarin,Warfarin,Warfarin,WARF5,006562,00056017275,5mg Tablet,5,mg,1,TAB,PO
2,2215524,6,107064,NaN,2175-06-11 00:00:00,2175-06-12 00:00:00,MAIN,Heparin Sodium,NaN,NaN,HEPAPREMIX,006522,00338055002,"25,000 unit Premix Bag","25,000",UNIT,1,BAG,IV
3,2216265,6,107064,NaN,2175-06-11 00:00:00,2175-06-12 00:00:00,BASE,D5W,NaN,NaN,HEPBASE,NaN,0,HEPARIN BASE,250,ml,250,ml,IV
4,2214773,6,107064,NaN,2175-06-11 00:00:00,2175-06-12 00:00:00,MAIN,Furosemide,Furosemide,Furosemide,FURO20,008208,00054829725,20mg Tablet,20,mg,1,TAB,PO


In [7]:
pres_ndc_name = pres_df[['NDC','DRUG']].drop_duplicates()

In [15]:
## find_top10 drugs
top10_NDC_ade = read_data(join(write_prefix,'top10_NDC_ade'),dtype={'NDC':str})
top10_NDCs = top10_NDC_ade[['NDC']]
top10_NDCs['index']=["drug_%d" % i for i in list(range(len(top10_NDCs)))]
top_drug_name = left_join(top10_NDCs, pres_ndc_name, 'NDC')
top_drug_name

,NDC,index,DRUG
0,00025152031,drug_0,Celebrex
1,00025152031,drug_0,Celecoxib
2,00025152031,drug_0,CeleBREX
3,00025152031,drug_0,celecoxib
4,00025152531,drug_1,Celecoxib
5,00025152531,drug_1,Celebrex
6,00025152531,drug_1,CeleBREX
7,00025152531,drug_1,celecoxib
8,24208063562,drug_2,Neomycin-Polymyxin-HC Otic Susp
9,00338114703,drug_3,Amino Acids 5%-Dextrose 15%


## Get Matrix of All Patients and All Items

In [5]:
diag_disease_all = diag_df[['SUBJECT_ID','ICD9_CODE','HADM_ID']].dropna().drop_duplicates()
pres_drug_all = pres_df[['SUBJECT_ID','NDC','HADM_ID']].dropna().drop_duplicates()
pres_drug_all = pres_drug_all[pres_drug_all['NDC']!='0']

In [6]:
print("Diagnoses: %d patients and %d diseases" 
      %(len(diag_disease_all.SUBJECT_ID.unique()),len(diag_disease_all.ICD9_CODE.unique())))
print("Prescriptions: %d patients and %d diseases" 
      %(len(pres_drug_all.SUBJECT_ID.unique()),len(pres_drug_all.NDC.unique())))

Diagnoses: 46517 patients and 6984 diseases
Prescriptions: 39357 patients and 4203 diseases


In [7]:
## randomly select one HADM_ID for each patient
diag_hadm_all = inner_join(diag_disease_all[['SUBJECT_ID','HADM_ID']].drop_duplicates(),\
                           pres_drug_all[['SUBJECT_ID','HADM_ID']].drop_duplicates(),'SUBJECT_ID')
diag_hadm_all = diag_hadm_all[diag_hadm_all['HADM_ID_x']==diag_hadm_all['HADM_ID_y']]
diag_hadm_all.head()

,SUBJECT_ID,HADM_ID_x,HADM_ID_y
16,109,172335,172335
36,109,173633,173633
88,109,126055,126055
123,109,128755,128755
165,109,131345,131345


In [8]:
size = 1        # sample size
## ramdonly get a sample hadm_id from each patient's record
fn = lambda obj: obj.loc[np.random.choice(obj.index, size),:]
hadm_all = diag_hadm_all.groupby('SUBJECT_ID', as_index=False).apply(fn)['HADM_ID_x']

In [ ]:
diag_hadm = diag_disease_all[diag_disease_all['HADM_ID'].isin(hadm_all)][['SUBJECT_ID','ICD9_CODE']]
pres_hadm = pres_drug_all[pres_drug_all['HADM_ID'].isin(hadm_all)][['SUBJECT_ID','NDC']]
diag_hadm.head()

In [ ]:
diag_hadm['VALUE']=1
pres_hadm['VALUE']=1

In [ ]:
diag_matrix = diag_hadm.pivot(index='SUBJECT_ID', columns='ICD9_CODE', values='VALUE').fillna(0).reset_index()
diag_matrix.head()

In [ ]:
pres_matrix = pres_hadm.pivot(index='SUBJECT_ID', columns='NDC', values='VALUE').fillna(0).reset_index()
pres_matrix.head()

In [ ]:
print("diag_shape: %s \npres_shape: %s"%(diag_matrix.shape,pres_matrix.shape))

In [ ]:
write2file(diag_matrix,join(write_prefix,"diag_matrix"))
write2file(pres_matrix,join(write_prefix,"pres_matrix"))

## Sampling

In [ ]:
diag_disease_all = diag_df[['ICD9_CODE']].dropna().drop_duplicates()
pres_drug_all = pres_df[['NDC']].dropna().drop_duplicates()
sample_disease_1000 = diag_disease_all.sample(n=1000, random_state=2019)
sample_drug_1000 = pres_drug_all.sample(n=1000, random_state=2019)

In [ ]:
diag_df_1000 = diag_df[diag_df['ICD9_CODE'].isin(sample_disease_1000['ICD9_CODE'])]
pres_df_1000 = pres_df[pres_df['NDC'].isin(sample_drug_1000['NDC'])]

write2file(diag_df_1000,join(write_prefix,"diag_df_1kdisease"))
write2file(pres_df_1000,join(write_prefix,"pres_df_1kdrug"))

In [ ]:
## select common 1000 users after filter 1000 drugs and diseases
common_user_all = inner_join(diag_df_1000[['SUBJECT_ID']].drop_duplicates(),\
                             pres_df_1000[['SUBJECT_ID']].drop_duplicates(),'SUBJECT_ID')
sample_user_1000 = common_user_all.sample(n=1000, random_state=2019)
diag_df_1k_1k = diag_df_1000[diag_df_1000['SUBJECT_ID'].isin(sample_user_1000['SUBJECT_ID'])]
pres_df_1k_1k = pres_df_1000[pres_df_1000['SUBJECT_ID'].isin(sample_user_1000['SUBJECT_ID'])]

In [ ]:
write2file(diag_df_1k_1k,join(write_prefix,"diag_df_1k_1k"))
write2file(pres_df_1k_1k,join(write_prefix,"pres_df_1k_1k"))

## Diag_df_1000 -> patients*clusters  *  cluster*elements

In [ ]:
diag_df_1k_1k = read_data(join(write_prefix,"diag_df_1k_1k"))
pres_df_1k_1k = read_data(join(write_prefix,"pres_df_1k_1k"))

In [ ]:
diag_df_1k_1k.head()

In [ ]:
## randomly select one HADM_ID for each patient
diag_hadm_all = inner_join(diag_df_1k_1k[['SUBJECT_ID','HADM_ID']].drop_duplicates(),\
                           pres_df_1k_1k[['SUBJECT_ID','HADM_ID']].drop_duplicates(),'SUBJECT_ID')
diag_hadm_all = diag_hadm_all[diag_hadm_all['HADM_ID_x']==diag_hadm_all['HADM_ID_y']]
diag_hadm_all.head()

In [ ]:
size = 1        # sample size
fn = lambda obj: obj.loc[np.random.choice(obj.index, size),:]
sample_hadm_1k = diag_hadm_all.groupby('SUBJECT_ID', as_index=False).apply(fn)['HADM_ID_x']

In [ ]:
diag_df_1k_1k_hadm = diag_df_1k_1k[diag_df_1k_1k['HADM_ID']\
                                   .isin(sample_hadm_1k)][['SUBJECT_ID','ICD9_CODE']].drop_duplicates()
diag_df_1k_1k_hadm['VALUE']=1

In [ ]:
diag_matrix = diag_df_1k_1k_hadm.pivot(index='SUBJECT_ID', columns='ICD9_CODE', values='VALUE').fillna(0).reset_index()
print(diag_matrix.shape)
diag_matrix.head()

In [ ]:
pres_df_1k_1k_hadm = pres_df_1k_1k[pres_df_1k_1k['HADM_ID']\
                                   .isin(sample_hadm_1k)][['SUBJECT_ID','NDC']].drop_duplicates()
pres_df_1k_1k_hadm['VALUE']=1


In [ ]:
pres_matrix = pres_df_1k_1k_hadm.pivot(index='SUBJECT_ID', columns='NDC', values='VALUE').fillna(0).reset_index()
print(pres_matrix.shape)
pres_matrix.head()


In [ ]:
write2file(diag_matrix,join(write_prefix,"diag_matrix"))
write2file(pres_matrix,join(write_prefix,"pres_matrix"))

## Reading matrix data

In [ ]:
## Select top10 drugs in sider4
sideffect_prefix = '/data/liu/adverse_events'
ade_df = read_data(join(sideffect_prefix, 'ndc_icd9_side_effects'), dtype={'NDC':str, 'ICD_CODE':str})
ade_df.head()

In [3]:
## OP1: All columns
# diag_raw_matrix = read_data(join(write_prefix, "diag_matrix"))
# pres_raw_matrix = read_data(join(write_prefix,"pres_matrix"))
# diag_raw_matrix.head()

# diag_patient, diag_matrix = diag_raw_matrix.iloc[:,0], diag_raw_matrix.iloc[:,1:]
# pres_patient, pres_matrix = pres_raw_matrix.iloc[:,0], pres_raw_matrix.iloc[:,1:]

diag_matrix = read_data(join(write_prefix, "diag_matrix"))
pres_matrix = read_data(join(write_prefix,"pres_matrix"))


In [27]:
## OP2: Top a few columns
diag_matrix_filtered = read_data(join(write_prefix,"diag_matrix_filtered"))
pres_matrix_filtered = read_data(join(write_prefix,"pres_matrix_filtered"))

## Validation

In [41]:
mimic_disease = read_data(join(read_prefix,"D_ICD_DIAGNOSES"),dtype={'ICD9_CODE':str})
mimic_disease[mimic_disease['ICD9_CODE'].isin(matched_disease)]

,ROW_ID,ICD9_CODE,SHORT_TITLE,LONG_TITLE
3561,3945,36221,Retrolental fibroplasia,Retrolental fibroplasia
5016,5396,4589,Hypotension NOS,"Hypotension, unspecified"
6009,6013,53081,Esophageal reflux,Esophageal reflux
13137,12986,7852,Cardiac murmurs NEC,Undiagnosed cardiac murmurs


In [50]:
# mimic_drugs = read_data(join(read_prefix,"PRESCRIPTIONS"),dtype={'NDC':str})[['NDC','DRUG']].drop_duplicates()
 mimic_drugs[mimic_drugs['NDC'].isin(top10_NDC_ade['NDC'])]

,NDC,DRUG
1394,00338114403,Amino Acids 4.25% W/ Dextrose 5%
3989,00264934155,Starter PN D10
4952,00264934155,Starter PN D5
30497,61958070101,Emtricitabine-Tenofovir
42287,00338114703,Amino Acids 5%-Dextrose 15%
69573,00025152531,Celecoxib
86309,00025152531,Celebrex
170619,61958070101,Emtricitabine-Tenofovir (Truvada)
208002,00025152031,Celebrex
505887,49702020218,LaMIVudine-Zidovudine (Combivir)


## Draft

In [ ]:
def test_lda(diag_params, pres_params, n_comp =2):
    
#     n_comp = 2
    ## doc_topic_prior, control the sparsity of probability
    diag_topic, diag_word = diag_params
    lda = LatentDirichletAllocation(n_components=n_comp, random_state=2019,\
                                    doc_topic_prior=diag_topic,topic_word_prior=diag_word)
    diag_Z = lda.fit_transform(diag_matrix.iloc[:,1:]) 
    # print(diag_Z.shape)
#     print("The first 5 rows of diagnoses_Z:\n %s"%diag_Z[:5])

    pres_topic, pres_word = pres_params
    lda_2 = LatentDirichletAllocation(n_components=n_comp, random_state=2019,\
                                      doc_topic_prior=pres_topic,topic_word_prior=pres_word)
    pres_Z = lda_2.fit_transform(pres_matrix.iloc[:,1:]) 
    # print(pres_Z.shape)
#     print("The first 5 rows of prescriptions_Z:\n %s"%pres_Z[:5])

    for i in range(n_comp):
        l = []
        for j in range(n_comp):
            sim_i_j = cosine_sim(diag_Z[:,i],pres_Z[:,j])
    #         print(sim_i_j)
            l = l + [sim_i_j]
        current_range = np.max(l)-np.min(l)
        print("Similarity between diag_%d and pres_j(j∈[0,%d): %s"%(i,n_comp, l))
        print("Max:%f  Range:%f arg_max:%d"%(np.max(l),current_range,np.argmax(l)))
        
    return diag_Z, pres_Z

In [ ]:
test_lda([0.1,0.001],[0.1,0.001],2)

In [ ]:
for n_comp in range(2,11):
    print("=============================================================\nn_comp: %d"%n_comp)
    test_lda([0.1,0.001],[0.1,0.001],n_comp)    

In [ ]:
## fix params for diagnoses, tune params for prescriptions
## increase by 0.1
for param in np.arange(0.2,1,0.2):
    print("pres_params: %s"%([param,param*0.01]))
    test_lda([0.1,0.001],[param,param*0.01],6)
## decrease by 0.01
for param in np.arange(0.01,0.1,0.02):
    print("pres_params: %s"%([param,param*0.01]))
    test_lda([0.1,0.001],[param,param*0.01],6) 

In [ ]:
diag_Y = lda.components_
diag_Y